```python
eosapi.gen_transaction(actions, expiration, reference_block_id)
```

```python
wallet.sign_transaction(trx, public_keys, chain_id)
```

```python
eosapi.push_transaction(trx, compress=0)
```

In [1]:
import json
import init
from uuoskit import uuosapi
from uuoskit import wasmcompiler
init.init()
test_account1 = init.test_account1

In [ ]:
'''
{'private': '5K3QY8YrtmT6K5VSSvTGaCm4g5xaxKo6yEHQ6nh1qn4ZYJptxAk',
 'public': 'EOS5AgSKfz3nbujRCJVeNv5ybBuMmhoQEUNQMQRWv1GaQAvxjBLq3'}
 
{'private': '5KhWH7iRi7BaFvGKbhbyNTXK7rXSG77SmCDcJJ1W6hpS5ysNd8T',
 'public': 'EOS67NHbUDvMhcqL4a4YyPEJZ784tZ38WfEhz6cBQ5RYbhf5KmjNc'}
'''

In [2]:
from uuoskit import wallet
wallet.import_key('test', '5K3QY8YrtmT6K5VSSvTGaCm4g5xaxKo6yEHQ6nh1qn4ZYJptxAk')
wallet.import_key('test', '5KhWH7iRi7BaFvGKbhbyNTXK7rXSG77SmCDcJJ1W6hpS5ysNd8T')

In [4]:
a = uuosapi.get_public_key('5KhWH7iRi7BaFvGKbhbyNTXK7rXSG77SmCDcJJ1W6hpS5ysNd8T', True)
b = uuosapi.get_public_key('5K3QY8YrtmT6K5VSSvTGaCm4g5xaxKo6yEHQ6nh1qn4ZYJptxAk', True)
a, b

('EOS67NHbUDvMhcqL4a4YyPEJZ784tZ38WfEhz6cBQ5RYbhf5KmjNc',
 'EOS5AgSKfz3nbujRCJVeNv5ybBuMmhoQEUNQMQRWv1GaQAvxjBLq3')

In [5]:
a = ['EOS67NHbUDvMhcqL4a4YyPEJZ784tZ38WfEhz6cBQ5RYbhf5KmjNc',
    'EOS5AgSKfz3nbujRCJVeNv5ybBuMmhoQEUNQMQRWv1GaQAvxjBLq3']
a.sort()
a

['EOS5AgSKfz3nbujRCJVeNv5ybBuMmhoQEUNQMQRWv1GaQAvxjBLq3',
 'EOS67NHbUDvMhcqL4a4YyPEJZ784tZ38WfEhz6cBQ5RYbhf5KmjNc']

In [6]:
a = {
    "account": test_account1,
    "permission": "sayhello",
    "parent": "active",
    "auth": {
        "threshold": 2,
        "keys": [
            {
                "key": "PUB_K1_5AgSKfz3nbujRCJVeNv5ybBuMmhoQEUNQMQRWv1GaQAw1U9sW6",
                "weight": 1
            },
            {
                "key": "PUB_K1_67NHbUDvMhcqL4a4YyPEJZ784tZ38WfEhz6cBQ5RYbhf2EBVcv",
                "weight": 1
            }
        ],
        "accounts": [],
        "waits": []
    }
}
r = uuosapi.push_action('eosio', 'updateauth', a, {test_account1:'active'})
print('done!')

done!


In [ ]:
args = {
    'account':test_account1,
    'code':test_account1,
    'type':'sayhello',
    'requirement':'sayhello'
}
r = uuosapi.push_action('eosio', 'linkauth', args, {test_account1:'active'})

In [8]:
uuosapi.get_account(test_account1)

{'account_name': 'helloworld11',
 'head_block_num': 8077723,
 'head_block_time': '2021-11-15T05:28:03.000',
 'privileged': False,
 'last_code_update': '2021-11-15T05:16:12.000',
 'created': '2021-02-07T17:59:33.000',
 'core_liquid_balance': '100000838.5901 EOS',
 'ram_quota': 15730017,
 'net_weight': '83716500642',
 'cpu_weight': '334875932576',
 'net_limit': {'used': 25089,
  'available': '27480957608712',
  'max': '27480957633801',
  'last_usage_update_time': {'slot': 230089756},
  'current_used': 25084},
 'cpu_limit': {'used': 14878,
  'available': '39306674412853',
  'max': '39306674427731',
  'last_usage_update_time': {'slot': 230089756},
  'current_used': 14875},
 'ram_usage': 176519,
 'permissions': [{'perm_name': 'active',
   'parent': 'owner',
   'required_auth': {'threshold': 1,
    'keys': [{'key': 'EOS6AjF6hvF7GSuSd4sCgfPKq5uWaXvGM2aQtEUCwmEHygQaqxBSV',
      'weight': 1}],
    'accounts': [{'permission': {'actor': 'helloworld11',
       'permission': 'eosio.code'},
      '

In [9]:
test_account1

'helloworld11'

In [11]:
code = r'''
package main

import "github.com/uuosio/chain"

func main() {
    receiver, _, _ := chain.GetApplyArgs()
    chain.RequireAuth2(receiver, chain.NewName("sayhello"))
    //chain.RequireAuth2(receiver, chain.NewName("owner"))    
    chain.Println("hello,world")
}
'''

code, abi = uuosapi.compile(test_account1, code, src_type=2)
uuosapi.deploy_code(test_account1, code, vm_type=0)
print('done!')

done!


In [11]:
from uuoskit import wallet
wallet.get_public_keys()

['EOS67NHbUDvMhcqL4a4YyPEJZ784tZ38WfEhz6cBQ5RYbhf5KmjNc',
 'EOS6AjF6hvF7GSuSd4sCgfPKq5uWaXvGM2aQtEUCwmEHygQaqxBSV',
 'EOS7sPDxfw5yx5SZgQcVb57zS1XeSWLNpQKhaGjjy2qe61BrAQ49o',
 'EOS89jesRgvvnFVuNtLg4rkFXcBg2Qq26wjzppssdHj2a8PSoWMhx',
 'EOS5AgSKfz3nbujRCJVeNv5ybBuMmhoQEUNQMQRWv1GaQAvxjBLq3']

In [12]:
r = uuosapi.push_action(test_account1, 'sayhello', b'hello,world', {test_account1:'sayhello'})
print(r['processed']['action_traces'][0]['console'])

hello,world



In [12]:
action = [test_account1, 'sayhello', b'hello,world', {test_account1:'sayhello'}]
info = uuosapi.get_info()
reference_block_id = info['last_irreversible_block_id']
chain_id = info['chain_id']
trx = uuosapi.gen_transaction([action], 60*60, reference_block_id, chain_id)
print(trx)

{"expiration":"2021-11-15T10:16:42","ref_block_num":21367,"ref_block_prefix":324289947,"max_net_usage_words":0,"max_cpu_usage_ms":0,"delay_sec":0,"context_free_actions":[],"actions":[{"account":"helloworld11","name":"sayhello","authorization":[{"actor":"helloworld11","permission":"sayhello"}],"data":"68656c6c6f2c776f726c64"}],"transaction_extensions":[]}


In [27]:
uuosapi.get_account(test_account1)['permissions'][2]

{'perm_name': 'sayhello',
 'parent': 'active',
 'required_auth': {'threshold': 2,
  'keys': [{'key': 'EOS5AgSKfz3nbujRCJVeNv5ybBuMmhoQEUNQMQRWv1GaQAvxjBLq3',
    'weight': 1},
   {'key': 'EOS67NHbUDvMhcqL4a4YyPEJZ784tZ38WfEhz6cBQ5RYbhf5KmjNc',
    'weight': 1}],
  'accounts': [],
  'waits': []}}

### sign with private key of EOS67NHbUDvMhcqL4a4YyPEJZ784tZ38WfEhz6cBQ5RYbhf5KmjNc

In [13]:
# a = ['EOS67NHbUDvMhcqL4a4YyPEJZ784tZ38WfEhz6cBQ5RYbhf5KmjNc',
#     'EOS5AgSKfz3nbujRCJVeNv5ybBuMmhoQEUNQMQRWv1GaQAvxjBLq3']

import hashlib
import json
public_keys = ['EOS67NHbUDvMhcqL4a4YyPEJZ784tZ38WfEhz6cBQ5RYbhf5KmjNc',]
info = uuosapi.get_info()
signed_trx = wallet.sign_transaction(trx, public_keys, info['chain_id'])
sign1 = signed_trx['signatures']
sign1

['SIG_K1_KksBjPqfy566KvZa9Zcm4VxMxJaycmqBbySi4MikxQyxTwQtALz7k9Tr5tUVWUcAHhPFUBKAgfmZJkxENotPzgMmXQgAy5']

In [5]:
import hashlib
info = uuosapi.get_info()
chain_id = bytes.fromhex(info['chain_id'])

packed_trx = uuosapi.pack_transaction(trx)
packed_trx = bytes.fromhex(packed_trx)
h = hashlib.sha256()
h.update(chain_id)
h.update(packed_trx)
h.update(b'\x00'*32)
wallet.sign_digest(h.hexdigest(), 'EOS67NHbUDvMhcqL4a4YyPEJZ784tZ38WfEhz6cBQ5RYbhf5KmjNc')

'SIG_K1_KhjN7E4x3DEFkkaowTK8F4eXG4hEzwqvpW7jL4w3ht6ENUz2GDs1McpfEJj7CHXjRuGcS8omBqsGnbdcZYvUmWAYhpiBTR'

### sign with private key of EOS5AgSKfz3nbujRCJVeNv5ybBuMmhoQEUNQMQRWv1GaQAvxjBLq3

In [6]:
signed_trx

{'signatures': ['SIG_K1_KhjN7E4x3DEFkkaowTK8F4eXG4hEzwqvpW7jL4w3ht6ENUz2GDs1McpfEJj7CHXjRuGcS8omBqsGnbdcZYvUmWAYhpiBTR'],
 'compression': 'none',
 'packed_context_free_data': '',
 'packed_trx': '353392615a535dbb474e000000000110428a97721aa36a0000003446d5bcc10110428a97721aa36a0000003446d5bcc10b68656c6c6f2c776f726c6400'}

In [14]:
public_keys = ['EOS5AgSKfz3nbujRCJVeNv5ybBuMmhoQEUNQMQRWv1GaQAvxjBLq3']
info = uuosapi.get_info()
signed_trx = wallet.sign_transaction(trx, public_keys, info['chain_id'])
sign2 = signed_trx['signatures']
sign2

['SIG_K1_Km6tm6epeMHWtzk7MxifPEy9LidW3Zt6qtVRdyAHWBBnWJy68FeKySVcVBK9SmQUba91ZySVs7Q88p4fj7kfN6qnNXvVy2']

In [15]:
signed_trx['signatures'] = sign1 + sign2
signed_trx['signatures'].sort()

In [16]:
r = uuosapi.push_transaction(signed_trx)
print(r['processed']['action_traces'][0]['console'])

hello,world

